# Aggregate route typology by route-direction
* If we selected a primary and secondary typology, **ignoring** the service frequency, would we be ok?
* 80% of the route-dir combinations have just max 2 significant typologies (over 10% of the route is that typology)

In [1]:
import geopandas as gpd
import pandas as pd

from shared_utils import rt_dates
from segment_speed_utils.project_vars import SCHED_GCS

from update_vars import CONFIG_DICT, analysis_date_list

FILE = CONFIG_DICT["route_typologies_file"]

route_dir_cols = [
    "schedule_gtfs_dataset_key", 
    "route_id", "direction_id", 
    "common_shape_id", "route_name", "route_meters"
]

analysis_date = rt_dates.DATES["mar2024"]

In [2]:
df = pd.read_parquet(
    f"{SCHED_GCS}{FILE}_long_{analysis_date}.parquet")

In [3]:
df2 = (df.groupby(route_dir_cols + ["typology"])
       .agg({"pct_typology": "sum"})
       .reset_index()
      )

In [4]:
df2 = df2.assign(
    obs = (df2
       .sort_values(
           route_dir_cols + ["pct_typology"], 
            ascending=[True for i in route_dir_cols] + [False])
       .groupby(route_dir_cols)
       .cumcount() + 1
      )
)

In [5]:
# 80% of the route-dir combinations have just max 2 significant typologies
df2.obs.value_counts(), df2.obs.value_counts(normalize=True)

(1    3610
 2    2207
 3    1000
 4     199
 Name: obs, dtype: int64,
 1    0.514538
 2    0.314567
 3    0.142531
 4    0.028364
 Name: obs, dtype: float64)